In [1]:
cd ..

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [126]:
from sklearn.feature_extraction.text import CountVectorizer
import polars as pl
from pathlib import Path
from src.utils.indexing import load_query_result
import polars.selectors as cs
import numpy as np

In [287]:
def get_values(list_values):
    s = "    ".join(list_values) + "    "
    return [s]

In [288]:
def prepare_table(table_path):
    table = pl.read_parquet(table_path)
    table_name = table_path.stem
    res = []
    for col in table.select(cs.string()).columns:
        values = "    ".join([_ for _  in table[col].to_list() if _ is not None])
        key = f"{table_name}__{col}"
        res.append((key, values))
    return res

In [289]:
cv = CountVectorizer(token_pattern=r"(?u)(<[\S]+>)[ ]{4}")

In [290]:
base_path = "data/source_tables/"


In [291]:
tab_name = "us-accidents-yadl.parquet"
tab_path = Path(base_path, tab_name)
df = pl.read_parquet(tab_path)

In [292]:
values = get_values(df["col_to_embed"].to_list())
cv.fit(values)

CountVectorizer(token_pattern='(?u)(<[\\S]+>)[ ]{4}')

In [294]:
len(cv.vocabulary_)

5222

In [295]:
[_ for _ in unique_source.to_list() if _.lower() not in cv.vocabulary_]

[]

In [296]:
left_on="col_to_embed"
right_on="isLocatedIn"
unique_source = df[left_on].unique()
unique_cand = df_cand[right_on].unique()

s1 = set(unique_source.to_list())
s2 = set(unique_cand.to_list())
print(len(s1.intersection(s2)) / len(s1))


0.10877058598238223


In [109]:
len(s1)

5222

In [147]:
base_path = Path("data/source_tables/")

In [229]:
rr = []
for pth in base_path.glob("*.parquet"):
    rr += prepare_table(pth)

In [164]:
X = cv.transform([_[1] for _ in rr])

In [218]:
sum_res = X.sum(axis=1).ravel()

In [230]:
s_index = sum_res.argsort()

In [225]:
keys = np.array([_[0] for _ in rr])

In [251]:
np.asarray(s_index).squeeze()

array([  0,  96,  97,  98,  99, 100, 101,  95, 102, 104, 105, 106, 108,
       109, 110, 103,  94,  93,  92,  76,  77,  78,  79,  80,  81,  82,
        83,  85,  86,  87,  88,  89,  90,  91, 111, 149, 112, 114, 135,
       136, 137, 138, 139, 140, 134, 141, 143, 144, 145, 146, 147, 148,
       142, 133, 132, 131, 116, 117, 118, 119, 120, 121, 122, 123, 124,
       125, 126, 127, 128, 129, 130, 113,  74,  75,  72,  20,  21,  22,
        23,  24,  25,  27,  28,  29,  30,  31,  32,  33,  34,  35,  19,
        73,  18,  16,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
        11,  12,  13,  14,  15,  17,  37,  36,  39,  57,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  56,  38,
        55,  53,  40,  41,  42,  43,  44,  45,  54,  46,  47,  48,  49,
        50,  51,  52, 150,  26, 115, 107,  84])